In [81]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link><main>
<article class="day-desc"><h2>--- Day 24: Electromagnetic Moat ---</h2><p>The CPU itself is a large, black building surrounded by a bottomless pit. Enormous metal tubes extend outward from the side of the building at regular intervals and descend down into the void. There's no way to cross, but you need to get inside.</p>
<p>No way, of course, other than building a <em>bridge</em> out of the magnetic components strewn about nearby.</p>
<p>Each component has two <em>ports</em>, one on each end.  The ports come in all different types, and only matching types can be connected.  You take an inventory of the components by their port types (your puzzle input). Each port is identified by the number of <em>pins</em> it uses; more pins mean a stronger connection for your bridge. A <code>3/7</code> component, for example, has a type-<code>3</code> port on one side, and a type-<code>7</code> port on the other.</p>
<p>Your side of the pit is metallic; a perfect surface to connect a magnetic, <em>zero-pin port</em>. Because of this, the first port you use must be of type <code>0</code>. It doesn't matter what type of port you end with; your goal is just to make the bridge as strong as possible.</p>
<p>The <em>strength</em> of a bridge is the sum of the port types in each component. For example, if your bridge is made of components <code>0/3</code>, <code>3/7</code>, and <code>7/4</code>, your bridge has a strength of <code>0+3 + 3+7 + 7+4 = 24</code>.</p>
<p>For example, suppose you had the following components:</p>
<pre><code>0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
</code></pre>
<p>With them, you could make the following valid bridges:</p>
<ul>
<li><code>0/1</code></li>
<li><code>0/1</code>--<code>10/1</code></li>
<li><code>0/1</code>--<code>10/1</code>--<code>9/10</code></li>
<li><code>0/2</code></li>
<li><code>0/2</code>--<code>2/3</code></li>
<li><code>0/2</code>--<code>2/3</code>--<code>3/4</code></li>
<li><code>0/2</code>--<code>2/3</code>--<code>3/5</code></li>
<li><code>0/2</code>--<code>2/2</code></li>
<li><code>0/2</code>--<code>2/2</code>--<code>2/3</code></li>
<li><code>0/2</code>--<code>2/2</code>--<code>2/3</code>--<code>3/4</code></li>
<li><code>0/2</code>--<code>2/2</code>--<code>2/3</code>--<code>3/5</code></li>
</ul>
<p>(Note how, as shown by <code>10/1</code>, order of ports within a component doesn't matter. However, you may only use each port on a component once.)</p>
<p>Of these bridges, the <em>strongest</em> one is <code>0/1</code>--<code>10/1</code>--<code>9/10</code>; it has a strength of <code>0+1 + 1+10 + 10+9 = <em>31</em></code>.</p>
<p><em>What is the strength of the strongest bridge you can make</em> with the components you have available?</p>
</article>


In [82]:
from collections import deque
from functools import cache
from typing import Generator
from more_itertools import minmax


example = """
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
"""


def parse_components(s: str) -> Generator[tuple[int, int], None, None]:
    return (map(int, l.split("/")) for l in s.strip().splitlines())


def get_components(s: str) -> dict[int, set[int]]:
    components = defaultdict(set)
    for mi, ma in parse_components(s):
        components[mi].add(ma)
        components[ma].add(mi)
    return components


def strongest_bridge(s: str) -> int:
    def dfs(fr: int) -> int:
        max_strenght = 0
        for to in componenents[fr]:
            if minmax(fr, to) not in seen:
                seen.add(minmax(fr, to))
                max_strenght = max(max_strenght, fr + to + dfs(to))
                seen.remove(minmax(fr, to))

        return max_strenght

    componenents = get_components(s)
    seen = set()
    return dfs(0)


print(f"Example : {strongest_bridge(example)} should be 31")

Example : 31 should be 31


In [83]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

print(f"Part I: {strongest_bridge(puzzle)}")

Part I: 1868


<link href="style.css" rel="stylesheet"></link><main>
...<main>

<p>Your puzzle answer was <code>1868</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The bridge you've built isn't long enough; you can't <span title="Who do you think you are, Mario?">jump the rest of the way</span>.</p>
<p>In the example above, there are two longest bridges:</p>
<ul>
<li><code>0/2</code>--<code>2/2</code>--<code>2/3</code>--<code>3/4</code></li>
<li><code>0/2</code>--<code>2/2</code>--<code>2/3</code>--<code>3/5</code></li>
</ul>
<p>Of them, the one which uses the <code>3/5</code> component is stronger; its strength is <code>0+2 + 2+2 + 2+3 + 3+5 = <em>19</em></code>.</p>
<p><em>What is the strength of the longest bridge you can make?</em> If you can make multiple bridges of the longest length, pick the <em>strongest</em> one.</p>
</article>

</main>


In [84]:
from more_itertools import first


def longest_strongest_bridge(s: str) -> int:
    def dfs(fr: int) -> tuple[int, int]:
        max_strenght, max_length = 0, 0
        for to in componenents[fr]:
            if minmax(fr, to) not in seen:
                seen.add(minmax(fr, to))
                w, l = dfs(to)
                if l > max_length:
                    max_strenght = fr + to + w
                    max_length = l
                elif l == max_length:
                    max_strenght = max(max_strenght, fr + to + w)
                seen.remove(minmax(fr, to))

        return max_strenght, max_length + 1

    componenents = get_components(s)
    seen = set()
    return first(dfs(0))


print(f"Example : {longest_strongest_bridge(example)} should be 19")

Example : 19 should be 19


In [85]:
print(f"Part II: {longest_strongest_bridge(puzzle)}")

Part II: 1841


<link href="style.css" rel="stylesheet"></link><main>
<main>

<p>Your puzzle answer was <code>1841</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
